# Obtención de Dataset

Pasos:
1. Colocar 4 arauco en las esquinas.
2. Medir y calibrar la cámara.
3. Detectar el robot con un 5to arauco distinto.
4. Detectar muros/contornos con threshold.

# Prueba de detección de Aruco Marker

In [1]:
import cv2 as cv
import numpy as np

# Cámara:
cap = cv.VideoCapture(0)

# Diccionarios y detectores Aruco Markers:
aruco_dict = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_100)
parameters = cv.aruco.DetectorParameters()
detector = cv.aruco.ArucoDetector(aruco_dict, parameters)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de ArUco
    corners, ids, rejected = detector.detectMarkers(frame)

    if ids is not None:
        cv.aruco.drawDetectedMarkers(frame, corners, ids)

        for i, corner in enumerate(corners):
            c = corner[0]
            # Coordenadas del centro
            x_corner = int((c[:,0].mean()))
            y_center = int((c[:,1].mean()))
            cv.circle(frame, (x_corner, y_center), 5, (0,0,255), -1)

            print(f"Marker {ids[i]} en píxeles: ({x_corner},{y_center})")

    cv.imshow("Frame", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

# Calibración de Cámara

Instrucciones de uso:

* Colocar en chess_size las dimensiones del tablero de ajedrez (es decir, la cantida de cuadrados en x e y)
* Colocar el tamaño de cada casilla en mm. Como son cuadradas por definición, solo se requiere el largo.


*  Se obtienen las variables: ret, mtx, dist, rvecs, tvecs

ret: Error de retención. (RMS de error general de la cámara).

mtx: Matriz de la cámara. (Matriz de la cámara. 3x3. Indica parámetros propios como el focal lenght).

dist: Coeficientes de distorción. **IMPORTANTE** (Array de 5 valores para corrección de la cámara. 3 de distorción radial y 2 de distorción tangencial).

rvecs: Vector de rotación. (Matriz de rotación de la imagen. Permite corregir la rotación).	**IRRELEVANTE**

tvecs: Vector de traslación. (Matriz de traslación de la imagen. Permite corregir la posición de la imagen).**IRRELEVANTE**


In [ ]:
import cv2
import numpy as np
import glob
import sys
import os

###### VARIABLES ###########################################################
chess_size = (9,6)					# Dimensiones de tablero patrón
chess_square_lenght = 30.0  		# En mm

calib_img_path = 'calibration_images'
test_img_path = 'test_images'

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)




#########################################################################################################
# Vector de posiciones de esquinas:
corner_size = (chess_size[0]-1, chess_size[1]-1)    #Las funciones utilizan las esquinas internas, no la cant de cuadrados
objp = np.zeros((corner_size[0]*corner_size[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:corner_size[0],0:corner_size[1]].T.reshape(-1,2)
objp *= chess_square_lenght

objpoints = [] # puntos 3D en el mundo real
imgpoints = [] # puntos 2D en la imagen

images = glob.glob(f'{calib_img_path}/*.jpg')

if not images:
    print("No hay imagenes.")
    sys.exit()
if len(images) < 10:
    print("Advertencia. Pocas imágenes. Se recomiendan 10 o más en distintos ángulos")

for fname in images:
    # print(f"Probando foto {fname}")
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)		# Lo pasa a escala de grises para mejorar
    ret, corners = cv2.findChessboardCornersSB(gray, corner_size, None)
    
    if ret == True:
        # print(f"Encontrado en {fname}")
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        cv2.drawChessboardCorners(img, corner_size, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

if not ret:
    print("No se encontró nada.")
    sys.exit()

# Calibración
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img = cv2.imread(images[0])
calib_res = (img.shape[1], img.shape[0])
print("Resolución usada para calibrar:", calib_res[0], "x", calib_res[1])

print("Matriz intrínseca:\n", mtx)
print("Coeficientes de distorsión:\n", dist)
np.savez("calibration_values", mtx=mtx, dist=dist, calib_res=calib_res)

# Verificacion
images = glob.glob(f'{test_img_path}/*.jpg')

if not images:
    print("No hay imagenes para verificar.")
    sys.exit()

for fname in images:
    print(f"Corrigiendo {fname}")
    img = cv2.imread(fname)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # Elimina distorcion
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
    # Recorte de la imagen
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    name = os.path.basename(fname)
    cv2.imwrite(f"{test_img_path}/calibrated_test/{name}_calib.jpg", dst)
    print(f"{fname} corregido y guardado en test_iamges/{name}_calib.jpg")




Resolución usada para calibrar: 2560 x 1440
Matriz intrínseca:
 [[1.99856782e+03 0.00000000e+00 1.29475608e+03]
 [0.00000000e+00 2.00339165e+03 7.42746877e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Coeficientes de distorsión:
 [[-6.28943106e-01  1.04540327e+00  1.36106218e-03  2.96854243e-04
  -1.80793651e+00]]
Corrigiendo test_images\WIN_20251014_15_07_53_Pro.jpg
test_images\WIN_20251014_15_07_53_Pro.jpg corregido y guardado en test_iamges/WIN_20251014_15_07_53_Pro.jpg_calib.jpg
Corrigiendo test_images\WIN_20251014_15_07_55_Pro.jpg
test_images\WIN_20251014_15_07_55_Pro.jpg corregido y guardado en test_iamges/WIN_20251014_15_07_55_Pro.jpg_calib.jpg
Corrigiendo test_images\WIN_20251014_15_07_56_Pro.jpg
test_images\WIN_20251014_15_07_56_Pro.jpg corregido y guardado en test_iamges/WIN_20251014_15_07_56_Pro.jpg_calib.jpg
Corrigiendo test_images\WIN_20251014_15_07_58_Pro.jpg
test_images\WIN_20251014_15_07_58_Pro.jpg corregido y guardado en test_iamges/WIN_20251014_15_07_58_Pro.jp

In [3]:
import cv2
import numpy as np
import glob

def calib_camara(img_path, chess_dim = (9,6), chess_square_lenght = 30.0, show = True):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    corner_size = (chess_dim[0]-1, chess_dim[1]-1)    #Las funciones utilizan las esquinas internas, no la cant de cuadrados

    objp = np.zeros((corner_size[0]*corner_size[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:corner_size[0],0:corner_size[1]].T.reshape(-1,2)
    objp *= chess_square_lenght     # Grilla con todas las posiciones de las esquinas

    objpoints = [] # puntos 3D en el mundo real
    imgpoints = [] # puntos 2D en la imagen

    # Lista de imagenes
    images = []
    for ext in ('*.jpg', '*.png', '*.jpeg'):
        images.extend(glob.glob(os.path.join(img_path, ext)))


    if not images:
        print("No hay imagenes.")
        return None
    if len(images) < 10:
        print("Advertencia. Pocas imágenes. Se recomiendan 10 o más en distintos ángulos")

    for fname in images:
        # print(f"Probando foto {fname}")
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)		# Lo pasa a escala de grises para mejorar
        ret, corners = cv2.findChessboardCornersSB(gray, corner_size, None)
        
        if ret == True:
            print(f"Encontrado en {fname}")
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners2)

            if show:
                cv2.drawChessboardCorners(img, corner_size, corners2, ret)
                cv2.imshow('img', img)
                cv2.waitKey(500)
    if show:
        cv2.destroyAllWindows()

    if len(objpoints) == 0:
        print("No se detectaron esquinas en ninguna imagen.")
        return None

    # Resolucion utilizada para calibrar
    img = cv2.imread(images[0])                  # Nota = usa solo la primer imagen !!! 
    calib_res = (img.shape[1], img.shape[0])    # calib_res = (WIDHT, HEIGHT) 

    # Calibración
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist, rvecs, tvecs, calib_res


def save_calib_matrix(file_name, mtx, dist, calib_res):
    try:
        np.savez(f"{file_name}", mtx=mtx, dist=dist, calib_res=calib_res )
        print(f"Matrices guardadas en {file_name}.npz")
    except Exception as e:
        print(f"Error al guardar: {e}")



def calib_img(img, mtx, dist):
    h,  w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # Elimina distorcion
    undistorted = cv2.undistort(img, mtx, dist, None, newcameramtx)
 
    # Recorte de la imagen
    x, y, w, h = roi
    undistorted = undistorted[y:y+h, x:x+w]
    return undistorted, newcameramtx, roi


def load_calib_matrix(file):
    try:
        calib = np.load(f"{file}.npz")
        mtx, dist, calib_res = calib["mtx"], calib["dist"], calib['calib_res']
    except Exception as e:
        mtx, dist, calib_res = None, None, None
    return mtx, dist, calib_res

# Medición de distancia Aruco Marker

Prueba 1:


El marker size es 100mm pero no me devuelve el Z correcto. Me devuelve el doble. Con marker size de 50 funciona bien

In [ ]:
import cv2
import sys

###### VARIABLES ###########################################################
marker_size = 50  # mm

#######################################################################################
cap = cv.VideoCapture(0)

### Creación del diccionario y detector
aruco_dict = cv.aruco.getPredefinedDictionary(cv.aruco.DICT_4X4_100)
parameters = cv.aruco.DetectorParameters()
detector = cv.aruco.ArucoDetector(aruco_dict, parameters)

### Carga de la Matriz de calibracion
mtx, dist = load_calib_matrix("calibration_values")

if mtx is None or dist is None:
    print("Faltan valores de calibracion")
    sys.exit()


### Loop de lectura de video
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de ArUco. NOTA: La funcion detecta todos los ArUco en pantalla y los pasa como lista con los ID.
    # corners[i][0][corner_num] -> (xi, yi)
    # ids[i] -> id del marker i
    corners, ids, _ = detector.detectMarkers(frame)

    if ids is not None:
        # tvecs[i][0] -> (x,y,z)
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_size, mtx, dist)

        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for i in range(len(ids)):
            cv2.aruco.drawDetectedMarkers(frame, corners, ids)

            # Dibujar ejes
            cv2.drawFrameAxes(frame, mtx, dist, rvecs[i], tvecs[i], 50)
            
            # Mostrar distancia Z (en mm)
            x, y, z = tvecs[i][0]

            print(f"marcador {ids[i]} en (x:{x:.2f},y:{x:.2f},z:{z:.2f})")

            x_corner = int(corners[i][0][3][0])
            y_corner = int(corners[i][0][3][1])
            
            
            #print(f"Marker {ids[i]} a {distancia:.1f} mm de la cámara")

            cv2.putText(frame, f"Z: {z:.1f}",(x_corner, y_corner - 10),                       
                       cv.FONT_HERSHEY_SIMPLEX, 
                       0.8, 
                       (255, 0, 0), 
                       2, 
                       cv.LINE_AA)

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Prueba 2: (El código fue sacado del siguiente [link](https://visionbrick.com/measuring-depth-from-a-single-camera-using-aruco-markers-in-opencv/))

In [ ]:
import cv2
import numpy as np

""" 
load your camera’s intrinsic matrix and distortion coefficients from a YAML file
- camera_matrix is the 3×3 intrinsic parameters matrix.
- dist_coeffs holds lens distortion parameters.
- marker_length is the side length of the ArUco marker in meters.
"""
camera_matrix, dist_coeffs = load_calib_matrix("calibration_values")
marker_length = 100  # mm
 
"""
DICT_4X4_50 is a predefined dictionary of ArUco markers.
4x4 markers with 50 unique IDs.(16 bits per marker)
parameters are the default detection parameters.
"""
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
parameters = cv2.aruco.DetectorParameters()
 
# create a detector instance with default parameters.
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)
 
cap = cv2.VideoCapture(0)
 
num=0
 
while True:
    ret, frame = cap.read()
    if not ret:
        break
 
    # Detect markers in the frame with detectMarkers method
    corners, ids, _ = detector.detectMarkers(frame)
    """
    Example output of corners:
    (array([[[ 15., 339.],
        [ 91., 334.],
        [ 98., 404.],
        [ 19., 414.]]], dtype=float32),)
    """
 
    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
 
        # Define the 3D coordinates of the marker corners in the marker's coordinate system
        obj_points = np.array([
            [-marker_length / 2,  marker_length / 2, 0],
            [ marker_length / 2,  marker_length / 2, 0],
            [ marker_length / 2, -marker_length / 2, 0],
            [-marker_length / 2, -marker_length / 2, 0]
        ], dtype=np.float32)
 
         
        for marker_corners in corners:
            image_points = marker_corners[0].astype(np.float32)
 
            """
            solvePnP estimates the pose of a 3D object given its 3D points and corresponding 2D image points.
            It returns the rotation vector (rvec) and translation vector (tvec).
            """
            retval, rvec, tvec = cv2.solvePnP(obj_points, image_points, camera_matrix, dist_coeffs)
             
            if retval:
                # Draw the axis on the frame
                cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, rvec, tvec, 0.03)
                 
                # Extract the translation vector and calculate the distance
                x, y, z = tvec.flatten()
                distance = np.linalg.norm(tvec)
                
                # Print to terminal
                print(f"X={x:.3f} m, Y={y:.3f} m, Z(depth)={z:.3f} m, Distance={distance:.3f} m")
                
                # Display on frame with different colors
                org = (20, 40)  
                font = cv2.FONT_HERSHEY_SIMPLEX
                font_scale = 0.7
                thickness = 2
                cv2.putText(frame, f"X={x:.3f} mm", (org[0], org[1]), font, font_scale, (0, 255, 0), thickness, cv2.LINE_AA)
                cv2.putText(frame, f"Y={y:.3f} mm", (org[0], org[1]+30), font, font_scale, (255, 0, 0), thickness, cv2.LINE_AA)
                cv2.putText(frame, f"Depth={z:.3f} mm", (org[0], org[1]+60), font, font_scale, (0, 0, 255), thickness, cv2.LINE_AA)
                cv2.putText(frame, f"Dist={distance:.3f} mm", (org[0], org[1]+90), font, font_scale, (0, 255, 255), thickness, cv2.LINE_AA)
 
    # Display the frame with detected markers and pose estimatio
    cv2.imshow('ArUco Pose Estimation', frame)
      
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
cap.release()
cv2.destroyAllWindows()

# Rectificación de la foto

Utilizando 4 ArUco Markers se busca que el plano quede totalmente perpendicular. Este código debería ajustar eso y ver "desde arriba" al plano.

Se le conoce como **Homografía**.

In [17]:
import cv2
import numpy as np
import sys

## Variables:
calib_value_file = "calibration_values"

corner_ids = [15, 15, 15, 15]

plane_size = (360, 460)     # En mm
pixels_per_mm = 2           # Resolution


# Cálculo de puntos de esquinas (ejemplo: ([0 0], [0 100], [24 0], [24 100]) mm)
image_size = (plane_size[0]*pixels_per_mm,
              plane_size[1]*pixels_per_mm) 

real_points = np.array([
                [0, 0],                             # arriba izq
                [image_size[0]-1, 0],               # arriba der
                [image_size[0]-1, image_size[1]-1], # abajo der
                [0, image_size[1]-1]                # abajo izq
            ], dtype=np.float32)


## Código:
mtx, dist, calib_res = load_calib_matrix(calib_value_file)
calib_res = tuple(calib_res)

if mtx is None or dist is None:
    print("No hay matriz intrinseca y/o coeficientes de distorsion")
    sys.exit(1)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, calib_res[0])
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, calib_res[1])

# Detector de ArUco
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)

while True:
    ret, frame = cap.read() # Solo para el caso de video. Omitir con fotos.

    if not ret:
        break

    # Corrección y detección de ArUco Marker:
    calib_frame, _, _ = calib_img(frame, mtx, dist)

    # corners[i][0][corner_num] -> (xi, yi)
    # ids[i] -> id del marker i
    corners, ids, _ = detector.detectMarkers(calib_frame)

    if ids is not None:
        calib_frame = cv2.aruco.drawDetectedMarkers(calib_frame, corners, ids)       # Solo visual

        # Busqueda de corners:
        ids = ids.flatten()
        detected_centers = []

        for i, marker_id in enumerate(ids):
            # Obtengo centros solo de los corners ID:
            if marker_id in corner_ids:
                c = corners[i][0]
                center = c.mean(axis=0)
                detected_centers.append(center)
                cv2.circle(frame, (int(center[0]), int(center[1])), 5, (0,0,255), -1)  # Solo visual

        if len(detected_centers) == 4:
            detected_centers = np.array(detected_centers, dtype=np.float32)

            s = detected_centers.sum(axis=1)
            diff = np.diff(detected_centers, axis=1)

            top_left = detected_centers[np.argmin(s)]
            bottom_right = detected_centers[np.argmax(s)]
            top_right = detected_centers[np.argmin(diff)]
            bottom_left = detected_centers[np.argmax(diff)]

            detected_points = np.array([top_left, top_right, bottom_right, bottom_left], dtype=np.float32)

            # Corrección de la imagen: 
            H, _ = cv2.findHomography(detected_points, real_points)
            aligned = cv2.warpPerspective(calib_frame, H, image_size)
            print("Imagen alineada correctamente.")
            cv2.imshow("Vista Alineada", aligned)

    cv2.imshow("Camara", calib_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen alineada correctamente.
Imagen a

In [10]:
import cv2
import numpy as np
import glob
import os

def warpImage(calib_value_file, images_path, corner_ids, plane_size, pixels_per_mm):
    '''
    Realiza Homografía con 4 esquinas ArUco.
    calib_value_file: Nombre de archivo (sin extensión) de las matrices de distorsión.
    images_path: path a carpeta de imágenes que van a realizar el path.
    corner_ids: IDs de ArUco a identificar como esquinas.
    plane_size: Tamaño del plano de 4 esquinas.
    pixels_per_mm: Resolución pixeles --> mm.
    '''
    # Cálculo de puntos de esquinas (ejemplo: ([0 0], [0 100], [24 0], [24 100]) mm)
    image_size = (plane_size[0]*pixels_per_mm,
                plane_size[1]*pixels_per_mm) 

    real_points = np.array([
                    [0, 0],                             # arriba izq
                    [image_size[0]-1, 0],               # arriba der
                    [image_size[0]-1, image_size[1]-1], # abajo der
                    [0, image_size[1]-1]                # abajo izq
                ], dtype=np.float32)

    # Detector de ArUco
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
    parameters = cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)


    ## Dir de Salida
    out_path = os.path.join(images_path, "warped")
    os.makedirs(out_path, exist_ok=True)

    ## Código:
    mtx, dist, calib_res = load_calib_matrix(calib_value_file)
    calib_res = tuple(calib_res)

    if mtx is None or dist is None:
        print("No hay matriz intrinseca y/o coeficientes de distorsion")
        return

    images = get_image_paths(images_path)

    for fname in images:
        image = cv2.imread(fname)

        if (image.shape[1], image.shape[0]) != calib_res:
            print("WARNING: Resolucion de imagen diferente a la de calibracion\n")

        # Corrección y detección de ArUco Marker:
        calib_frame, _, _ = calib_img(image, mtx, dist)
        # corners[i][0][corner_num] -> (xi, yi)
        # ids[i] -> id del marker i
        
        gray_calib_img = cv2.cvtColor(calib_frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = detector.detectMarkers(gray_calib_img)
        frame_markers = calib_frame.copy()
        cv2.aruco.drawDetectedMarkers(frame_markers, corners, ids)
        cv2.imshow("Aruco detectados", frame_markers)
        cv2.waitKey(500)


        corner_coords = get_corners(corner_ids, corners, ids)
        if corner_coords is None or len(corner_coords) != 4:
            print('no hay coordenadas')
            continue

        # Corrección de la imagen: 
        H, _ = cv2.findHomography(corner_coords, real_points)
        aligned = cv2.warpPerspective(calib_frame, H, image_size)

        outname = os.path.join(out_path, os.path.basename(fname))
        cv2.imwrite(outname, aligned)
        print(f"Imagen {fname} alineada correctamente.\nGuardada en: {out_path}")

def get_corners(corner_ids, corners, ids):
    '''
    Devuelve un vector de 4 posiciones con los centros de cada esquina. 
    El resultado está ordenado [TOP LEFT, TOP RIGHT, BOTTOM RIGHT, BOTTOM LEFT]
    corner_ids: Id's a utilizar como esquina.
    corners: Vectos de esquinas de todos los ArUco obtenidos.
    ids: Id de cada ArUco enviado en corners.
    '''
    detected_centers = []
    corner_coords = []
    if ids is not None:
        # Busqueda de corners:
        ids = ids.flatten()

        for i, marker_id in enumerate(ids):
            # Obtengo centros solo de los corners ID:
            if marker_id in corner_ids:
                c = corners[i][0]
                center = c.mean(axis=0)
                detected_centers.append(center)

        if len(detected_centers) == 4:
            detected_centers = np.array(detected_centers, dtype=np.float32)

            s = detected_centers.sum(axis=1)
            diff = np.diff(detected_centers, axis=1)

            top_left = detected_centers[np.argmin(s)]
            bottom_right = detected_centers[np.argmax(s)]
            top_right = detected_centers[np.argmin(diff)]
            bottom_left = detected_centers[np.argmax(diff)]

            corner_coords = np.array([top_left, top_right, bottom_right, bottom_left], dtype=np.float32)

    return corner_coords
        
def get_image_paths(image_path):
    '''
    Devuelve una lista con todas las imágenes en una carpeta específica.
    image_path: Path de carpeta donde se buscan imágenes.
    '''
    images = []
    for ext in ('*.jpg', '*.png', '*.jpeg'):
        images.extend(glob.glob(os.path.join(image_path, ext)))
    
    return images


In [11]:
## Variables:
calib_value_file = "calibration_values"

corner_ids = [15, 15, 15, 15]

plane_size = (595, 420)     # En mm
pixels_per_mm = 2           # Resolution

warpImage(calib_value_file, 'warp_test', corner_ids, plane_size, pixels_per_mm)



Imagen warp_test\WIN_20251015_16_31_27_Pro.jpg alineada correctamente.
Guardada en: warp_test\warped
Imagen warp_test\WIN_20251015_16_31_28_Pro.jpg alineada correctamente.
Guardada en: warp_test\warped
Imagen warp_test\WIN_20251015_16_31_29_Pro.jpg alineada correctamente.
Guardada en: warp_test\warped
no hay coordenadas
no hay coordenadas
Imagen warp_test\WIN_20251015_16_31_44_Pro.jpg alineada correctamente.
Guardada en: warp_test\warped
no hay coordenadas


In [ ]:
### estimatePoseBoard para pasar a imagen sin distorsion




### Hacer threshold de color:
### -Pasar imagen a RGB2HSV
### -bitwise_or

In [12]:
def colorDetect(images_path, color_range, show=True):
    """
    Detecta regiones de un color específico en todas las imágenes dentro de 'images_path/warped'
    y guarda los resultados en 'images_path/color'.

    Parameters:
        images_path (str): Carpeta base donde están las imágenes originales.
        color_range (tuple): (lower_HSV, upper_HSV) con los límites del color a detectar.
        show (bool): Muestra los resultados visualmente (por defecto True).
    """

    warped_dir = os.path.join(images_path, "warped")
    color_dir = os.path.join(images_path, "color")
    os.makedirs(color_dir, exist_ok=True)

    images = []
    for ext in ('*.jpg', '*.png', '*.jpeg'):
        images.extend(glob.glob(os.path.join(warped_dir, ext)))

    if not images:
        print("No hay imágenes en la carpeta 'warped'.")
        return

    lower, upper = color_range

    for fname in images:
        print(f"Procesando {fname}")
        image = cv2.imread(fname)

        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower, upper)

        # Limpieza de ruido
        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # resultado a color
        result = cv2.bitwise_and(image, image, mask=mask)

        outname_mask = os.path.join(color_dir, os.path.basename(fname).replace(".", "_mask."))
        outname_result = os.path.join(color_dir, os.path.basename(fname).replace(".", "_color."))

        cv2.imwrite(outname_mask, mask)
        cv2.imwrite(outname_result, result)
        print(f"Guardado: {outname_mask} y {outname_result}")

        if show:
            cv2.imshow("Original", image)
            cv2.imshow("Detección", result)
            key = cv2.waitKey(500)
            if key == 27:  # ESC para salir
                break

    cv2.destroyAllWindows()


In [13]:
lower_blue = (100, 150, 50)
upper_blue = (130, 255, 255)

colorDetect("warp_test", (lower_blue, upper_blue))


Procesando warp_test\warped\WIN_20251015_16_31_27_Pro.jpg
Guardado: warp_test\color\WIN_20251015_16_31_27_Pro_mask.jpg y warp_test\color\WIN_20251015_16_31_27_Pro_color.jpg
Procesando warp_test\warped\WIN_20251015_16_31_28_Pro.jpg
Guardado: warp_test\color\WIN_20251015_16_31_28_Pro_mask.jpg y warp_test\color\WIN_20251015_16_31_28_Pro_color.jpg
Procesando warp_test\warped\WIN_20251015_16_31_29_Pro.jpg
Guardado: warp_test\color\WIN_20251015_16_31_29_Pro_mask.jpg y warp_test\color\WIN_20251015_16_31_29_Pro_color.jpg
Procesando warp_test\warped\WIN_20251015_16_31_44_Pro.jpg
Guardado: warp_test\color\WIN_20251015_16_31_44_Pro_mask.jpg y warp_test\color\WIN_20251015_16_31_44_Pro_color.jpg
